<a href="https://colab.research.google.com/github/prismvale/Podcast-highlights/blob/main/31oct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio

In [ ]:
def reverse_text(text):
    return text[::-1]

In [ ]:
import gradio as gr

# Frontend + backend together
demo = gr.Interface(
    fn=reverse_text,
    inputs="text",
    outputs="text",
    title="🪞 Text Reverser",
    description="Type something and watch it reverse instantly!"
)

demo.launch(share=True, quiet=True)

/usr/local/lib/python3.12/dist-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


Running on public URL: https://820afaf97c1a7260f8.gradio.live


In [ ]:
demo.close()


In [ ]:
!pip install openai-whisper yt-dlp webvtt-py transformers torch --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 12.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.0/176.0 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 57.0 MB/s eta 0:00:00


In [ ]:
# @title
# ==============================
# 🎥 AI-Powered Auto Clip Generator
# ==============================

import os
import re
import subprocess
import yt_dlp
import whisper
import webvtt
from transformers import pipeline

# ==============================
# 1️⃣ USER INPUT
# ==============================
url = input("Enter YouTube URL: ").strip()

# ==============================
# 🧹 CLEANUP OLD FILES
# ==============================
def cleanup_old_files():
    patterns = ["video.mp4", "video.en.vtt", "clip_*.mp4", "clip_*.vtt"]
    for pattern in patterns:
        for file in [f for f in os.listdir() if re.fullmatch(pattern.replace("*", ".*"), f)]:
            try:
                os.remove(file)
                print(f"🧹 Removed old file: {file}")
            except Exception as e:
                print(f"⚠️ Could not remove {file}: {e}")

cleanup_old_files()

video_file = "video.mp4"
sub_file = "video.en.vtt"

# ==============================
# 2️⃣ DOWNLOAD VIDEO
# ==============================
print("⬇️ Downloading video...")
ydl_opts = {"format": "mp4", "outtmpl": "video.%(ext)s"}
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])

# ==============================
# 3️⃣ GENERATE SUBTITLES USING WHISPER
# ==============================
print("🎤 Generating subtitles with Whisper...")
model = whisper.load_model("tiny")
result = model.transcribe(video_file)

with open(sub_file, "w", encoding="utf-8") as f:
    f.write("WEBVTT\n\n")
    for i, seg in enumerate(result["segments"]):
        start, end, text = seg["start"], seg["end"], seg["text"].strip()
        def sec_to_vtt(s):
            h = int(s // 3600)
            m = int((s % 3600) // 60)
            sec = s % 60
            return f"{h:02}:{m:02}:{sec:06.3f}"
        f.write(f"{i+1}\n{sec_to_vtt(start)} --> {sec_to_vtt(end)}\n{text}\n\n")

print("✅ Subtitles saved as video.en.vtt")



# ==============================
# 4️⃣ GROUP INTO SENTENCE CHUNKS
# ==============================
print("🧩 Creating text chunks...")
grouped_chunks = []
current_chunk, chunk_start, chunk_end = [], None, None
max_chunk_duration, min_chunk_duration = 25.0, 10.0

for caption in webvtt.read(sub_file):
    start_sec = sum(float(x) * 60 ** i for i, x in enumerate(reversed(caption.start.split(":"))))
    end_sec = sum(float(x) * 60 ** i for i, x in enumerate(reversed(caption.end.split(":"))))

    if chunk_start is None:
        chunk_start = start_sec

    sentences = [s.strip() for s in re.split(r'(?<=[.!?]) +', caption.text) if s.strip()]

    for sentence in sentences:
        current_chunk.append(sentence)
        chunk_end = end_sec

        # Only create chunk if max duration exceeded
        if chunk_start is not None and chunk_end is not None and (chunk_end - chunk_start >= max_chunk_duration):
            if grouped_chunks and (chunk_end - chunk_start < min_chunk_duration):
                # Merge with previous chunk safely
                grouped_chunks[-1]["end"] = chunk_end
                grouped_chunks[-1]["text"] += " " + " ".join(current_chunk)
            else:
                # Create a new chunk
                grouped_chunks.append({
                    "start": chunk_start,
                    "end": chunk_end,
                    "text": " ".join(current_chunk)
                })
            # Reset for next chunk
            current_chunk, chunk_start, chunk_end = [], None, None

# Handle leftover sentences at the end
if current_chunk:
    if chunk_start is not None and chunk_end is not None:
        if grouped_chunks and (chunk_end - chunk_start < min_chunk_duration):
            grouped_chunks[-1]["end"] = chunk_end
            grouped_chunks[-1]["text"] += " " + " ".join(current_chunk)
        else:
            grouped_chunks.append({
                "start": chunk_start,
                "end": chunk_end,
                "text": " ".join(current_chunk)
            })

print(f"✅ Total chunks created: {len(grouped_chunks)}")




# ==============================
# 5️⃣ ANALYZE EMOTION + SENTIMENT
# ==============================
print("🧠 Analyzing emotion and sentiment...")
sentiment_model = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")
emotion_model = pipeline("text-classification",
                         model="joeddav/distilbert-base-uncased-go-emotions-student",
                         return_all_scores=True)

scored_chunks = []
for chunk in grouped_chunks:
    text = chunk["text"]
    sentiment = sentiment_model(text[:512])[0]
    emotions = emotion_model(text[:512])[0]
    top_emotion = max(emotions, key=lambda x: x['score'])
    score = sentiment['score'] * 5
    if top_emotion['label'] in ["joy", "excitement", "anger", "sadness"]:
        score += top_emotion['score'] * 5
    scored_chunks.append({
        "start": chunk["start"],
        "end": chunk["end"],
        "text": text,
        "sentiment": sentiment['label'],
        "emotion": top_emotion['label'],
        "score": score
    })

# ==============================
# 6️⃣ SELECT TOP 3 CLIPS
# ==============================
scored_chunks = sorted(scored_chunks, key=lambda x: x["score"], reverse=True)
top3 = scored_chunks[:3]

print("\n🔥 Top 3 Engaging Moments:")
for i, c in enumerate(top3, 1):
    print(f"\n--- Clip {i} ---")
    print(f"Time: {c['start']:.2f}s → {c['end']:.2f}s")
    print(f"Emotion: {c['emotion']} | Sentiment: {c['sentiment']} | Score: {c['score']:.2f}")
    print(f"Text: {c['text'][:150]}...")

# ==============================
# 7️⃣ EXTRACT CLIPS + SUBTITLES
# ==============================
print("\n✂️ Extracting top 3 clips...")
captions = list(webvtt.read(sub_file))

def format_time(seconds):
    h = int(seconds // 3600)
    m = int((seconds % 3600) // 60)
    s = seconds % 60
    return f"{h:02d}:{m:02d}:{s:06.3f}".replace('.', ',')

for i, clip in enumerate(top3, 1):
    start, end = clip["start"], clip["end"]
    out_video, out_sub = f"clip_{i}.mp4", f"clip_{i}.vtt"

    # Extract video
    cmd = ["ffmpeg", "-y", "-i", video_file, "-ss", str(start), "-to", str(end), "-c", "copy", out_video]
    subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # Extract subtitles for the clip
    selected = []
    for c in captions:
        start_sec = sum(float(x) * 60 ** i for i, x in enumerate(reversed(c.start.split(":"))))
        end_sec = sum(float(x) * 60 ** i for i, x in enumerate(reversed(c.end.split(":"))))
        if start_sec >= start and end_sec <= end:
            new_start, new_end = start_sec - start, end_sec - start
            selected.append(f"{format_time(new_start)} --> {format_time(new_end)}\n{c.text}\n")

    with open(out_sub, "w", encoding="utf-8") as f:
        f.write("WEBVTT\n\n" + "\n".join(selected))

    print(f"✅ Saved: {out_video} + {out_sub}")

print("\n🎉 Done! Your top 3 clips and subtitles are ready.")


In [ ]:
# @title
!pip install openai-whisper yt-dlp webvtt-py transformers torch gradio --quiet

# ==============================
# 🎥 AI-Powered Auto Clip Generator (Colab + Gradio Frontend)
# ==============================
import os
import re
import subprocess
import yt_dlp
import whisper
import webvtt
import gradio as gr
from transformers import pipeline

# ==============================
# ⚙️ Core Logic Function
# ==============================
def process_youtube_video(url):
    try:
        # ---------- CLEANUP ----------
        patterns = ["video.mp4", "video.en.vtt", "clip_.*\.mp4", "clip_.*\.vtt"]
        for pattern in patterns:
            for file in [f for f in os.listdir() if re.fullmatch(pattern.replace("*", ".*"), f)]:
                try:
                    os.remove(file)
                except Exception:
                    pass

        video_file = "video.mp4"
        sub_file = "video.en.vtt"

        # ---------- DOWNLOAD ----------
        ydl_opts = {"format": "mp4", "outtmpl": "video.%(ext)s"}
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])

        # ---------- WHISPER ----------
        model = whisper.load_model("tiny")
        result = model.transcribe(video_file)

        with open(sub_file, "w", encoding="utf-8") as f:
            f.write("WEBVTT\n\n")
            for i, seg in enumerate(result["segments"]):
                start, end, text = seg["start"], seg["end"], seg["text"].strip()
                def sec_to_vtt(s):
                    h = int(s // 3600)
                    m = int((s % 3600) // 60)
                    sec = s % 60
                    return f"{h:02}:{m:02}:{sec:06.3f}"
                f.write(f"{i+1}\n{sec_to_vtt(start)} --> {sec_to_vtt(end)}\n{text}\n\n")

        # ---------- GROUP CHUNKS ----------
        grouped_chunks = []
        current_chunk, chunk_start, chunk_end = [], None, None
        max_chunk_duration, min_chunk_duration = 25.0, 10.0

        for caption in webvtt.read(sub_file):
            start_sec = sum(float(x) * 60 ** i for i, x in enumerate(reversed(caption.start.split(":"))))
            end_sec = sum(float(x) * 60 ** i for i, x in enumerate(reversed(caption.end.split(":"))))

            if chunk_start is None:
                chunk_start = start_sec

            sentences = [s.strip() for s in re.split(r'(?<=[.!?]) +', caption.text) if s.strip()]

            for sentence in sentences:
                current_chunk.append(sentence)
                chunk_end = end_sec
                if chunk_start is not None and chunk_end is not None and (chunk_end - chunk_start >= max_chunk_duration):
                    if grouped_chunks and (chunk_end - chunk_start < min_chunk_duration):
                        grouped_chunks[-1]["end"] = chunk_end
                        grouped_chunks[-1]["text"] += " " + " ".join(current_chunk)
                    else:
                        grouped_chunks.append({
                            "start": chunk_start,
                            "end": chunk_end,
                            "text": " ".join(current_chunk)
                        })
                    current_chunk, chunk_start, chunk_end = [], None, None

        if current_chunk:
            if chunk_start is not None and chunk_end is not None:
                if grouped_chunks and (chunk_end - chunk_start < min_chunk_duration):
                    grouped_chunks[-1]["end"] = chunk_end
                    grouped_chunks[-1]["text"] += " " + " ".join(current_chunk)
                else:
                    grouped_chunks.append({
                        "start": chunk_start,
                        "end": chunk_end,
                        "text": " ".join(current_chunk)
                    })

        # ---------- ANALYSIS ----------
        sentiment_model = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")
        emotion_model = pipeline("text-classification",
                                 model="joeddav/distilbert-base-uncased-go-emotions-student",
                                 return_all_scores=True)

        scored_chunks = []
        for chunk in grouped_chunks:
            text = chunk["text"]
            sentiment = sentiment_model(text[:512])[0]
            emotions = emotion_model(text[:512])[0]
            top_emotion = max(emotions, key=lambda x: x['score'])
            score = sentiment['score'] * 5
            if top_emotion['label'] in ["joy", "excitement", "anger", "sadness"]:
                score += top_emotion['score'] * 5
            scored_chunks.append({
                "start": chunk["start"],
                "end": chunk["end"],
                "text": text,
                "sentiment": sentiment['label'],
                "emotion": top_emotion['label'],
                "score": score
            })

        # ---------- SELECT TOP 3 ----------
        scored_chunks = sorted(scored_chunks, key=lambda x: x["score"], reverse=True)
        top3 = scored_chunks[:3]

        result_text = "🔥 **Top 3 Engaging Moments:**\n\n"
        #for i, c in enumerate(top3, 1):
            #result_text += f"### 🎬 Clip {i}\n"
            #result_text += f"- Time: {c['start']:.2f}s → {c['end']:.2f}s\n"
            #result_text += f"- Emotion: {c['emotion']} | Sentiment: {c['sentiment']} | Score: {c['score']:.2f}\n"
            #result_text += f"- Text: {c['text'][:150]}...\n\n"

        # ---------- EXTRACT CLIPS ----------
        captions = list(webvtt.read(sub_file))
        def format_time(seconds):
            h = int(seconds // 3600)
            m = int((seconds % 3600) // 60)
            s = seconds % 60
            return f"{h:02d}:{m:02d}:{s:06.3f}".replace('.', ',')

        video_files = []
        for i, clip in enumerate(top3, 1):
            start, end = clip["start"], clip["end"]
            out_video, out_sub = f"clip_{i}.mp4", f"clip_{i}.vtt"
            cmd = ["ffmpeg", "-y", "-i", video_file, "-ss", str(start), "-to", str(end), "-c", "copy", out_video]
            subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            video_files.append(out_video)

            selected = []
            for c in captions:
                start_sec = sum(float(x) * 60 ** i for i, x in enumerate(reversed(c.start.split(":"))))
                end_sec = sum(float(x) * 60 ** i for i, x in enumerate(reversed(c.end.split(":"))))
                if start_sec >= start and end_sec <= end:
                    new_start, new_end = start_sec - start, end_sec - start
                    selected.append(f"{format_time(new_start)} --> {format_time(new_end)}\n{c.text}\n")

            with open(out_sub, "w", encoding="utf-8") as f:
                f.write("WEBVTT\n\n" + "\n".join(selected))

        return result_text, video_files[0] if len(video_files) > 0 else None, video_files[1] if len(video_files) > 1 else None, video_files[2] if len(video_files) > 2 else None

    except Exception as e:
        return f"⚠️ Error: {e}", None, None, None

# ==============================
# 🎨 Gradio UI
# ==============================
demo = gr.Interface(
    fn=process_youtube_video,
    inputs=gr.Textbox(label="Enter YouTube URL"),
    outputs=[
        gr.Markdown(label="Results"),
        gr.Video(label="Clip 1"),
        gr.Video(label="Clip 2"),
        gr.Video(label="Clip 3")
    ],
    title="🎥 AI Auto Clip Generator",
    description="Automatically extracts top 3 emotional or engaging clips from a YouTube video using Whisper + NLP sentiment/emotion analysis.",
    allow_flagging="never"
)

demo.launch(debug=True)


<>:22: SyntaxWarning: invalid escape sequence '\.'
<>:22: SyntaxWarning: invalid escape sequence '\.'
<>:22: SyntaxWarning: invalid escape sequence '\.'
<>:22: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipython-input-98566741.py:22: SyntaxWarning: invalid escape sequence '\.'
  patterns = ["video.mp4", "video.en.vtt", "clip_.*\.mp4", "clip_.*\.vtt"]
/tmp/ipython-input-98566741.py:22: SyntaxWarning: invalid escape sequence '\.'
  patterns = ["video.mp4", "video.en.vtt", "clip_.*\.mp4", "clip_.*\.vtt"]
/usr/local/lib/python3.12/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://fadc52037670257f52.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


[youtube] Extracting URL: https://youtu.be/BYizgB2FcAQ?si=A7T4_qEFnpq-ph7a
[youtube] BYizgB2FcAQ: Downloading webpage
[youtube] BYizgB2FcAQ: Downloading android sdkless player API JSON
[youtube] BYizgB2FcAQ: Downloading tv client config
[youtube] BYizgB2FcAQ: Downloading player c6d7bdc9-main
[youtube] BYizgB2FcAQ: Downloading tv player API JSON
[youtube] BYizgB2FcAQ: Downloading web safari player API JSON


         player = https://www.youtube.com/s/player/c6d7bdc9/player_ias.vflset/en_US/base.js
         n = SK5lSexV7wl5b_99 ; player = https://www.youtube.com/s/player/c6d7bdc9/player_ias.vflset/en_US/base.js
         Please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U


[youtube] BYizgB2FcAQ: Downloading m3u8 information
[info] BYizgB2FcAQ: Downloading 1 format(s): 96
[download] Sleeping 5.00 seconds as required by the site...
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 635
[download] Destination: video.mp4
[download] 100% of  507.46MiB in 00:00:41 at 12.11MiB/s                  
[FixupM3u8] Fixing MPEG-TS in MP4 container of "video.mp4"


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Device set to use cpu
Device set to use cpu
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://fadc52037670257f52.gradio.live
